<a href="https://colab.research.google.com/github/PalepuRohith/Generative-Ai-/blob/main/Text_Summarizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
pip install transformers accelerate datasets


INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.3 MB/s eta 0:00:00


In [4]:
from datasets import load_dataset

multinews = load_dataset("multi_news", split ='test')

multi_news.py:   0%|          | 0.00/3.83k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

The repository for multi_news contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/multi_news.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


train.src.cleaned:   0%|          | 0.00/548M [00:00<?, ?B/s]

train.tgt:   0%|          | 0.00/58.8M [00:00<?, ?B/s]

...

test.src.cleaned:   0%|          | 0.00/69.0M [00:00<?, ?B/s]

test.tgt:   0%|          | 0.00/7.31M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/44972 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5622 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5622 [00:00<?, ? examples/s]

In [5]:
multinews.to_pandas()

,document,summary
0,GOP Eyes Gains As Voters In 11 States Pick Gov...,– It's a race for the governor's mansion in 11...
1,\n \n \n \n UPDATE: 4/19/2001 Read Richard Met...,– It turns out Facebook is only guilty of abou...
2,It's the Golden State's latest version of the ...,– Not a big fan of Southern California? Neithe...
3,The seed for this crawl was a list of every ho...,– Why did Microsoft buy Nokia's phone business...
4,After a year in which liberals scored impressi...,– The Supreme Court is facing a docket of high...
...,...,...
5617,Tweet with a location \n \n You can add locati...,– The traditional end-of-summit group photo at...
5618,Loic Venance/AFP/Getty Images \n \n The awards...,– Sofia Coppola scored a historic victory at t...
5619,(CNN) A federal criminal investigation into a ...,– The duck boat sinking that killed 17 on a Mi...
5620,An archive of the public statements deleted by...,– Note to tweeting politicians: Watch what you...


In [6]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("t5-small")

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [7]:
multinews = multinews.train_test_split(test_size=0.2)

In [10]:
prefix = "summarize:"
def process_function(examples):
  inputs = [ prefix + doc for doc in examples['document']]
  model_input = tokenizer(inputs, max_length = 1024, truncation =True)
  labels = tokenizer(text = examples['summary'], max_length = 128, truncation = True)
  model_input['labels'] = labels['input_ids']

  return model_input




In [11]:
tokenized_multi_news = multinews.map(process_function, batched = True)

Map:   0%|          | 0/4497 [00:00<?, ? examples/s]

Map:   0%|          | 0/1125 [00:00<?, ? examples/s]

In [14]:
from transformers import DataCollatorForSeq2Seq, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer, model = 't5-small')
model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [16]:
training_args = Seq2SeqTrainingArguments(
    output_dir = './results',
    eval_strategy = 'epoch',
    learning_rate = 2e-5,
    per_device_train_batch_size = 10,
    per_device_eval_batch_size = 10,
    weight_decay = 0.01,
    save_total_limit = 3,
    num_train_epochs = 10,
    fp16 = True
)

In [17]:
trainer = Seq2SeqTrainer(
    model = model,
    args = training_args,
    train_dataset = tokenized_multi_news['train'],
    eval_dataset = tokenized_multi_news['test'],
    tokenizer = tokenizer,
    data_collator = data_collator
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [18]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,2.848597
2,3.260600,2.788368
3,2.998000,2.758720
4,2.973700,2.742295


Epoch,Training Loss,Validation Loss
1,No log,2.848597
2,3.260600,2.788368
3,2.998000,2.758720
4,2.973700,2.742295
5,2.917500,2.731002
6,2.905100,2.726145
7,2.894100,2.719797
8,2.883400,2.716515
9,2.862800,2.714815
10,2.875400,2.713776


TrainOutput(global_step=4500, training_loss=2.9522985297309026, metrics={'train_runtime': 2800.3227, 'train_samples_per_second': 16.059, 'train_steps_per_second': 1.607, 'total_flos': 1.217264162439168e+16, 'train_loss': 2.9522985297309026, 'epoch': 10.0})

In [21]:
text = "Artificial Intelligence (AI) has long captured our collective imagination with its ability to mimic human-like intelligence and perform tasks that were once exclusive to the realm of human cognition. However, traditional AI systems have primarily focused on tasks like classification, prediction, and optimization, relying on large datasets to learn patterns and make decisions. While these systems have been incredibly useful, they are limited in their capacity to create truly novel content. Enter Generative AI, a burgeoning field that has been rapidly gaining traction in recent years. Generative AI represents a paradigm shift in the AI landscape, moving beyond mere recognition and prediction to actively creating new content autonomously. From generating lifelike images and music compositions to crafting compelling narratives and virtual environments, generative AI holds the promise of unlocking new levels of creativity and innovation across a wide range of domains. At its core, generative AI seeks to understand and model the underlying patterns within data, allowing machines to generate outputs that closely resemble the input they were trained on. This ability to synthesize new content has opened up exciting possibilities for artists, researchers, and businesses alike, ushering in a new era of exploration and discovery. The roots of generative AI can be traced back to the early days of neural networks and machine learning. However, significant advancements in deep learning techniques, coupled with the availability of massive datasets and computational resources, have propelled the field forward in recent years. One of the seminal developments in generative AI is the introduction of Generative Adversarial Networks (GANs) by Ian Goodfellow and his colleagues in 2014. GANs revolutionized the field by introducing a novel framework for training generative models through adversarial learning. By pitting a generator network against a discriminator network in a competitive game, GANs are able to generate highly realistic synthetic data, such as images and videos, that are indistinguishable from real data to human observers. Another influential approach in generative AI is Variational Autoencoders (VAEs), which were introduced around the same time as GANs. VAEs leverage variational inference techniques to learn a probabilistic latent space representation of the input data. This latent space can then be sampled to generate new data points that resemble the training data, making VAEs particularly well-suited for tasks like image generation and data imputation. In addition to GANs and VAEs, researchers have explored a myriad of other generative models and techniques, each with its own strengths and applications. Autoregressive models, such as Transformers, have emerged as powerful tools for generating sequential data, such as text and music, by modeling the conditional probability distribution of each element given the previous elements in the sequence. The seeds of generative AI were sown in the nascent days of neural networks and machine learning research. In the 1960s and 1970s, pioneers such as Marvin Minsky and Seymour Papert laid the groundwork for neural network theory, exploring the concept of perceptrons and their ability to learn from data. However, early efforts were hampered by computational limitations and a lack of sufficient data to train complex models."

In [23]:
input_ids = tokenizer(text, max_length = 1024, truncation = True, return_tensors ='pt').input_ids
input_ids = input_ids.to('cuda')

In [31]:
import torch

with torch.no_grad():
  if model.device.type == 'cuda':
    output = model.generate(input_ids, max_length =162, num_beams = 5)

summary_ids = output[0].tolist()

summary = tokenizer.decode(summary_ids, skip_special_tokens = True)
print(summary)

, and a new era of exploration and discovery. In the 1960s and 1970s, pioneers such as Marvin Minsky and Seymour Papert laid the groundwork for neural networks and machine learning research, exploring the concept of perceptrons and their ability to learn from data. In the 1960s and 1970s, pioneers such as Marvin Minsky and Seymour Papert laid the groundwork for neural network theory, exploring the concept of perceptrons and their ability to learn from data. In the 1960


In [32]:
ref_summary = """
Artificial Intelligence (AI) has long been used to mimic human-like intelligence and perform tasks, but traditional systems have limited capacity to create novel content. Generative AI, a rapidly gaining field, aims to create new content autonomously, generating lifelike images, music compositions, narratives, and virtual environments. This approach seeks to understand and model underlying patterns within data, allowing machines to generate outputs that closely resemble the input they were trained on. The roots of generative AI can be traced back to neural networks and machine learning, but advancements in deep learning techniques and the availability of massive datasets have propelled the field forward. GANs, introduced by Ian Goodfellow in 2014, revolutionized the field by introducing a framework for training generative models through adversarial learning. Variational Autoencoders (VAEs) were introduced around the same time, leveraging variational inference techniques to generate probabilistic latent space representations of input data. Other generative models and techniques, such as autoregressive models like Transformers, have emerged for generating sequential data."""

In [28]:
!pip install rouge-score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=f5405e8c20447cd5ee01ec79dcd798bd0d8d2386104e3e0ec56b7cb2ddd09592
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [33]:
from rouge_score import rouge_scorer
rouge = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
scores = rouge.score(summary, ref_summary)
print(scores)


{'rouge1': Score(precision=0.20245398773006135, recall=0.4230769230769231, fmeasure=0.27385892116182575), 'rouge2': Score(precision=0.030864197530864196, recall=0.06493506493506493, fmeasure=0.04184100418410042), 'rougeL': Score(precision=0.12269938650306748, recall=0.2564102564102564, fmeasure=0.16597510373443983)}


rouge 1 compare what is the maximum sequence matching in actual result
rouge 2 compares bigrams
rouge l is similar to rouge 1 but it is regardless of the sequence



In [35]:
model.save_pretrained("summarizer")